# Chapter 08 - Exercises

### Exercise 8.1

**Q**

The nonplanning method looks particularly poor in Figure 8.3 because it is a one-step method; a method using multi-step bootstrapping would do better. Do you think one of the multi-step bootstrapping methods from Chapter 7 could do as well as the Dyna method? Explain why or why not.

**A**

A similar corresponding case is using a nonplanning method with multi-step bootstrapping with $n = 51$, because the planning simulates 50 cases after the direct RL update, and would update the last 51 state-action values after each step (in the first episode, only the last 51 states visited before the end of the episode will be updated, because only the final reward is non-zero). 

In the second episode, while the planning method starts with only the last non terminal state updated, after each step it can update a previously visited states considering the actions taken, and while in the beginning it has little chance of choosing a useful state-action pair that takes to a next state that was already updated, as the steps go on it updates more and more cases, going fast toward states closest to the beginning.

In the same second episode, for the non-planing 51-step TD, updating the values using the Q-learning method (that uses the best state-action value after in the last step, or 0 for the terminal state), all the initial steps wouldn't be able to update any state-action values. Only after the agent reaches an already updated state, it would update the states of the last steps, so it might be faster than the first case, TD(0), but it would probably take longer than the planning method because it can only update the last 51 steps, while the planning method may even update a state that was not reached yet in the episode. 

Furthermore, because the value of n for the n-step method is high, the possibility of a high variance is higher (although, because this environment has all rewards 0 except the last, the variance may ending up not being so high). One thing that might cause a high variance is a state s1 close to the end, but the agent took a bad direction and went to a lot of states farther from the final state, and then went to the final state without passing from the indicated state s1, making it have a worse value than it should have (this is more difficult to happen in the planning case, that uses TD(0), because it only consider neighbor states, reducing considerably the chances like the above scenario to happen, even if it's actually possible to happen too).

So, using a 51-step TD with Q-learning (using the maximum state-action value in the last step) may converge faster than TD(0) (or not, due to the variance), and can update a number of states just like the planning method with 50 simulations per step, but because the updates need to follow the last 50 steps, and can not be benefited from updates from previous episodes if it has not reached a state already updated in a previous episode, it probably will need more episodes than the planning method (using less steps for the nonplanning method may actually help to converge faster, although it probably won't have the same performance as the planning method). This answer is just a speculation tough. A real experiment may give a more concrete explanation.

### Exercise 8.2 

**Q**

Why did the Dyna agent with exploration bonus, Dyna-Q+, perform better in the first phase as well as in the second phase of the blocking and shortcut experiments?

**A**

It was better in the second phase due to the reasons already stated  in the example: it explores more than the Dyna agent, so when the environment changed, it found the change faster (in the shortcut experiment, Dyna haven't found the shortcut even in the next 3000 time-steps, while Dyna-Q+ found it after about 1000 time-steps).

For the first phase, the reason is less obvious, but it has to do with the fact that Dyna-Q+ gives priority to action-states not tried before. Considering that both used $\epsilon$-greedy policies, which means that there's already a possibility of exploration at each time-step, Dyna-Q+ gives priority to unknown states, making it explore and find the terminal state faster (Dyna may end up repeating the same states several times before finding the path to the terminal state, at least in the first episode).

### Exercise 8.3 

**Q**

Careful inspection of Figure 8.5 reveals that the difference between Dyna-Q+ and Dyna-Q narrowed slightly over the first part of the experiment. What is the reason for this?

**A**

The Dyna-Q+ method explores more and consequently is able to find the best path faster, but that has a cost, and that cost is that it continues exploring even if the best path was already found. Both methods explore due to the soft policy used, but in the Dyna case that exploration should be minimal. This means that after the optimal path is found, if no changes happen to the environment, the Dyna method may end up being a bit better than Dyna-Q+ over time (more reward per time-step, on average).

### Exercise 8.4 (programming) 

**Q**

The exploration bonus described above actually changes the estimated values of states and actions. Is this necessary? Suppose the bonus $\kappa \sqrt{\tau}$ was used not in updates, but solely in action selection. That is, suppose the action selected was always that for which $Q(S_t, a) + \kappa \sqrt{\tau(S_t, a)}$ was maximal. Carry out a gridworld experiment that tests and illustrates the strengths and weaknesses of this alternate approach.

**A**

### Exercise 8.5 

**Q**

How might the tabular Dyna-Q algorithm shown on page 164 be modified to handle stochastic environments? How might this modification perform poorly on changing environments such as considered in this section? How could the algorithm be modified to handle stochastic environments and changing environments?

**A**

In a stochastic environment, given a state and an action done in this state, instead of a fixed next state and reward, there's a distribution of probabilities for each possible next state and reward. To handle such an environment, each next state and reward can be given a weight, incremented by 1 every time that case happen, with the planning done based on this distribution: the model in this case returns the distribution probability, and the simulation is done picking one of the possibilities based on its probability.

This might perform poorly when the environment changes due to the weights from the previous environment, and lots of real experiments will be required to make the new (actually, updated) weights outweight the (now incorrect) previous weights. To handle both scenarios, before incrementing the weight of the next state and reward given a state and an action, it can multiply all the weights of the distribution of the given state and action by a discount factor smaller than 1 to reduce the overall impact of the previous cases, but higher than 0 to not discard them completely.

Of course, making the agent explore more, especially state-action pairs not used for a long time, can also be seen as a possible solution, in which case the Dyna-Q method would actually become Dyna-Q+.